In [14]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import GetHashedMorganFingerprint
import torch
from torch import nn
from torch.utils.data import Dataset, random_split, DataLoader

In [110]:
def get_morgan_fp(smi):
    mol = Chem.MolFromSmiles(smi)
    fp = GetHashedMorganFingerprint(mol, radius=2, nBits=2048, useChirality=True)
    return np.array(fp.ToList())

class LipoDataset(Dataset):
    def __init__(self):
        df = pd.read_csv('lipo.csv')
        self.labels = df['exp'].to_numpy()
        self.fps = np.vectorize(get_morgan_fp, otypes=[np.ndarray])(df.smiles.to_numpy())

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        return np.ndarray.astype(self.fps[idx], np.float64), self.labels[idx]

In [111]:
lipo_dataset = LipoDataset()
size_dataset = len(lipo_dataset)

In [112]:
m, n = next(iter(lipo_dataset))
n.dtype

dtype('float64')

In [113]:
size_train = int(np.rint(size_dataset * 0.8))
size_valid = int(np.rint(size_dataset * 0.1))
size_test = size_dataset - (size_train + size_valid)

In [114]:
train_dataset, valid_dataset, test_dataset = random_split(lipo_dataset, [size_train, size_valid, size_test])

In [115]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

In [148]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [149]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2048, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=1, bias=True)
  )
)


In [150]:
learning_rate = 1e-3
epochs = 10
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [151]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X.float()).double()
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 20 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.float()).double()
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    correct /= size
    print(f"Avg loss: {test_loss:>8f} \n")

In [152]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(valid_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 7.092487  [    0/ 3360]
loss: 5.467043  [ 1280/ 3360]
loss: 5.227544  [ 2560/ 3360]
Avg loss: 4.600107 

Epoch 2
-------------------------------
loss: 4.465962  [    0/ 3360]
loss: 2.973189  [ 1280/ 3360]
loss: 2.014531  [ 2560/ 3360]
Avg loss: 2.080664 

Epoch 3
-------------------------------
loss: 1.746188  [    0/ 3360]
loss: 1.541743  [ 1280/ 3360]
loss: 1.761625  [ 2560/ 3360]
Avg loss: 1.695774 

Epoch 4
-------------------------------
loss: 1.751262  [    0/ 3360]
loss: 1.363201  [ 1280/ 3360]
loss: 1.410647  [ 2560/ 3360]
Avg loss: 1.699437 

Epoch 5
-------------------------------
loss: 1.763343  [    0/ 3360]
loss: 1.476708  [ 1280/ 3360]
loss: 1.452687  [ 2560/ 3360]
Avg loss: 1.714463 

Epoch 6
-------------------------------
loss: 1.974288  [    0/ 3360]
loss: 1.983840  [ 1280/ 3360]
loss: 1.672390  [ 2560/ 3360]
Avg loss: 1.663034 

Epoch 7
-------------------------------
loss: 1.677062  [    0/ 3360]
loss: 1.608085  [ 1280/ 